# An Introduction to LROSE

<img align="right" width="200" height="200" src="../images/colette_rose.jpg">
<img align="right" width="200" height="200" src="../images/LROSE_logo_small.png">
    
The Lidar Radar Open Software Environment (LROSE) is an National Science Foundation (NSF) supported project to develop common software for the Lidar, Radar, and Profiler community based on collaborative, open source development.  The core package is being jointly developed by Colorado State University (CSU) and the Earth Observing Laboratory at the NSF National Center for Atmospheric Research (NSF NCAR/EOL). The current LROSE release is called Colette (a versatile climbing rose). 

More information on LROSE can be found on the [LROSE wiki](http://wiki.lrose.net/index.php/Main_Page).

## Notebook Summary

This notebook will cover the following:

1. LROSE Overview
2. Basic LROSE application command line usage
3. Example LROSE application parameter files
4. Basic data inspection
5. Converting a file to CfRadial
6. Simple gridding
7. Basic plotting

## LROSE Overview

LROSE encompasses six key toolsets that define a core lidar/radar workflow: Convert, Display, QC, Grid, Echo, and Winds. Colette focuses on high-quality, well-tested, well-maintained and well-documented key applications as ‘building blocks’, allowing users to assemble trusted, reproducible workflows to accomplish more complex scientific tasks.

### Installation

LROSE is available for download through [GitHub](https://github.com/NCAR/lrose-core/). Installation is supported for Linux (source, packages) and Mac OS (source, Homebrew). Conda-forge development is underway.

### Initialize python

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import datetime
import fsspec
import glob
import pytz
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
from matplotlib.lines import Line2D
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.geodesic as cgds
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy import feature as cfeature
import shapely
import netCDF4 as nc
import pyart

# Set directory variable to call LROSE
os.environ['LROSE_DIR'] = '/usr/local/lrose/bin'
base_dir = os.environ['BASE_DIR']

# Set the URL and path to access the data on the cloud
URL = "https://js2.jetstream-cloud.org:8001/"
path = f"pythia/radar/erad2024"

fs = fsspec.filesystem("s3", anon=True, client_kwargs=dict(endpoint_url=URL))

fs.glob(f"{path}/*")

files = fs.glob(f"{path}/*") #### FIX PATH
local_files = [
    fsspec.open_local(
        f"simplecache::{URL}{i}", s3={"anon": True}, filecache={"cache_storage": "."}
    )
    for i in files
]


In [ ]:
!pwd

## Notes on LROSE Parameter files

All LROSE applications have a detailed parameter file, which is read in at startup. The parameters allow the user to control the processing in the LROSE apps. To generate a default parameter file, you use the -print_params option for the app.

For example, for RadxConvert you would use:

```
  RadxConvert -print_params > RadxConvert.nexrad
```

and then edit RadxConvert.nexrad appropriately.

At runtime you would use:

```
  RadxConvert -params RadxConvert.nexrad ... etc ...
```

## View the RadxConvert parameter file

Note that we can use environment variables in the parameter files.

Environment variables are inserted using the format:

```
  $(env_var_name)
```

For example:

```
  input_dir = "$(NEXRAD_DATA_DIR)/raw/$(RADAR_NAME)";
```


In [ ]:
# View the param file
!cat $BASE_DIR/params/erad/RadxConvert.nexrad

## List the CfRadial files created by RadxConvert

In [ ]:
# List the CfRadial files created by RadxConvert
!ls -R ${NEXRAD_DATA_DIR}/cfradial/moments/K*/20*

In [ ]:
!${LROSE_DIR}/RadxPrint -h

In [ ]:
!${LROSE_DIR}/RadxPrint -print_params

In [ ]:
!${LROSE_DIR}/RadxPrint -print_params > RadxPrint_params

## Inspect the data

In [ ]:
!${LROSE_DIR}/RadxPrint -f /path/to/cfradial 

In [ ]:
!${LROSE_DIR}/RadxPrint -summary -f /path/to/cfradial | head -200

## Format Conversion

In [ ]:
# Convert raw nexrad data to cfradial for 3 NEXRAD radars
!${LROSE_DIR}/RadxConvert -f /path/to/cfradial -outdir /path/


## Simple Gridding

In [ ]:
!${LROSE_DIR}/Radx2Grid -f /path/to/cfradial -outdir /path/

## Simple Display

### Py-Art

In [ ]:
# Read CfRadial file into radar object
filePathRate = os.path.join(nexradDataDir, "cfradial/rate/KGLD/20210706/cfrad.20210706_220003.963_to_20210706_220439.770_KGLD_SUR.nc")
rate_kgld = pyart.io.read_cfradial(filePathRate)
rate_kgld.info('compact')

In [ ]:
# Plot results of RadxRate

displayRate = pyart.graph.RadarDisplay(rate_kgld)
figRate = plt.figure(1, (12, 10))

# DBZ (input)

axDbz = figRate.add_subplot(221)
displayRate.plot_ppi('DBZ', 0, vmin=-32, vmax=64.,
                    axislabels=("x(km)", "y(km)"),
                    colorbar_label="DBZ")
displayRate.plot_range_rings([50, 100, 150, 200])
displayRate.plot_cross_hair(200.)

# KDP (computed)

axKdp = figRate.add_subplot(222)
displayRate.plot_ppi('KDP', 0, vmin=0, vmax=2.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="KDP (deg/km)",
    cmap="rainbow")
displayRate.plot_range_rings([50, 100, 150, 200])
displayRate.plot_cross_hair(200.)

# RATE_HYBRID (computed)

axHybrid = figRate.add_subplot(223)
displayRate.plot_ppi('RATE_HYBRID', 0, vmin=0, vmax=50.,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="RATE_HYBRID(mm/hr)",
    cmap = "rainbow")
displayRate.plot_range_rings([50, 100, 150, 200])
displayRate.plot_cross_hair(200.)

# NCAR PID (computed)

axPID = figRate.add_subplot(224)
displayRate.plot_ppi('PID', 0,
    axislabels=("x(km)", "y(km)"),
    colorbar_label="PID",
    cmap = "rainbow")
displayRate.plot_range_rings([50, 100, 150, 200])
displayRate.plot_cross_hair(200.)

pid_cbar = displayRate.cbs[3]
pid_cbar.set_ticks([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
pid_cbar.set_ticklabels(['cld-drops', 'drizzle', 'lt-rain', 'mod-rain', 'hvy-rain', 'hail', 'rain/hail', 'sm-hail', 'gr/rain', 'dry-snow', 'wet-snow', 'ice', 'irreg-ice', 'slw', 'insects', '2nd-trip', 'clutter'])

figRate.tight_layout()

plt.show()

### Gridded Data

In [ ]:
# Create map for plotting lat/lon grids
def new_map(fig):
    
    ## Create projection centered on data
    proj = ccrs.PlateCarree()

    ## New axes with the specified projection:
    ax = fig.add_subplot(1, 1, 1, projection=proj)
    
    ## Set extent the same as radar mosaic
    ax.set_extent([minLonMosaic, maxLonMosaic, minLatMosaic, maxLatMosaic])

    ## Add grid lines & labels:
    gl = ax.gridlines( crs=ccrs.PlateCarree()
                     , draw_labels=True
                     , linewidth=1
                     , color='lightgray'
                     , alpha=0.5, linestyle='--'
                     ) 
    gl.top_labels = False
    gl.left_labels = True
    gl.right_labels = False
    gl.xlines = True
    gl.ylines = True
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size': 8, 'weight': 'bold'}
    gl.ylabel_style = {'size': 8, 'weight': 'bold'}
    
    return ax

In [ ]:
# Plot column-max reflectivity
figDbzComp = plt.figure(figsize=(8, 8), dpi=150)
axDbzComp = new_map(figDbzComp)
plt.imshow(dbzPlaneMax,
            cmap='pyart_Carbone42',
            interpolation = 'bilinear',
            origin = 'lower',
            extent = (minLonMosaic, maxLonMosaic, minLatMosaic, maxLatMosaic))
axDbzComp.add_feature(cfeature.BORDERS, linewidth=0.5, edgecolor='black')
axDbzComp.add_feature(cfeature.STATES, linewidth=0.3, edgecolor='brown')
#axDbzComp.coastlines('10m', 'darkgray', linewidth=1, zorder=0)
plt.colorbar(label="DBZ", orientation="vertical", shrink=0.5)
plt.title("Radar mosaic column-max DBZ: " + startTimeStr)